In [1]:
import gradio as gr
import datetime as dt
import random
import string
from pathlib import Path
from typing import List, Dict

import pandas as pd
import spacy
from email.message import EmailMessage
from email import policy
from email.parser import BytesParser
from transformers import pipeline

# ───── Configuration
OUT_DIR = Path("emails")
OUT_DIR.mkdir(exist_ok=True)

SENDERS = [
    "billing@company.com", "hr@startup.io", "support@webapp.org",
    "sales@techcorp.com", "info@example.org"
]
RECIPIENTS = ["user@example.com"]
SUBJECTS = [
    "Invoice overdue", "Payment confirmation",
    "Job application for Data Analyst", "Issue with logging in",
    "New product launch update", "Resume submission for HR position",
    "Need support with account", "Sales inquiry for bulk pricing"
]
BODIES = [
    "Hello, I would like to confirm that I’ve made the payment for the last invoice.",
    "There’s an issue with my login credentials. Can you help me fix it?",
    "I’m applying for the Data Analyst position. Please find my resume attached.",
    "We are launching a new product next month. Here's the marketing plan.",
    "Can I get a quote for purchasing 500 units of your product?",
    "Please help me reset my password. I can't access my account.",
    "Attached is my resume for the HR Assistant role.",
    "Reminder: Your invoice from last month is still unpaid."
]
ZERO_LABELS = [
    "billing", "technical support", "human resources",
    "marketing", "sales", "general"
]


def generate_random_email(save_path: Path) -> None:
    msg = EmailMessage()
    msg["From"] = random.choice(SENDERS)
    msg["To"] = random.choice(RECIPIENTS)
    msg["Subject"] = random.choice(SUBJECTS)
    msg["Date"] = dt.datetime.utcnow().strftime("%a, %d %b %Y %H:%M:%S +0000")
    uid = "".join(random.choices(string.ascii_uppercase + string.digits, k=8))
    msg.set_content(f"{random.choice(BODIES)}\n\n--ID:{uid}--")
    save_path.write_bytes(bytes(msg))


def parse_eml(path: Path) -> Dict[str, str]:
    msg = BytesParser(policy=policy.default).parsebytes(path.read_bytes())
    body_part = msg.get_body(preferencelist=("plain",))
    body = body_part.get_content() if body_part else ""
    return {
        "file": path.name,
        "from": msg.get("from", ""),
        "subject": msg.get("subject", ""),
        "date": msg.get("date", ""),
        "body": body,
    }


def rule_intent(text: str) -> str:
    txt = text.lower()
    if any(k in txt for k in ("invoice", "payment")):
        return "billing"
    if any(k in txt for k in ("resume", "job")):
        return "human resources"
    if any(k in txt for k in ("issue", "problem", "password", "support", "login")):
        return "technical support"
    if any(k in txt for k in ("quote", "pricing", "purchase")):
        return "sales"
    if any(k in txt for k in ("launch", "marketing", "product")):
        return "marketing"
    return "general"


# Load once
nlp = spacy.load("en_core_web_sm")
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


def process_emails(n: int, enable_zero: bool) -> pd.DataFrame:
    paths = []
    for i in range(n):
        path = OUT_DIR / f"email_{i+1}.eml"
        generate_random_email(path)
        paths.append(path)

    data: List[Dict[str, str]] = []
    for p in paths:
        rec = parse_eml(p)
        rec["intent_rule"] = rule_intent(rec["body"])

        doc = nlp(rec["body"])
        rec["persons"] = ", ".join(e.text for e in doc.ents if e.label_ == "PERSON")

        # if enable_zero:
        #     result = classifier(rec["body"], ZERO_LABELS, multi_label=False)
        #     rec["intent_zero"] = result["labels"][0]
        #     rec["zero_score"] = round(result["scores"][0], 3)

        data.append(rec)

    df = pd.DataFrame(data)
    display_cols = ["file", "from", "subject", "intent_rule", "persons"]
    if enable_zero:
        display_cols += ["intent_zero", "zero_score"]

    return df[display_cols]


# ───── Gradio UI
demo = gr.Interface(
    fn=process_emails,
    inputs=[
        gr.Slider(minimum=1, maximum=10, value=3, step=1, label="Number of Random Emails"),
        gr.Checkbox(label="Enable Zero-Shot Classification (Hugging Face)")
    ],
    outputs=gr.Dataframe(label="Parsed & Categorized Emails"),
    title="📧 Email Parser & Categorizer",
    description="Generate random emails, parse them, extract intent using rules and optionally classify them using Hugging Face zero-shot model."
)

if __name__ == "__main__":
    demo.launch()


c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
